# LEVERAGING MACHINE LEARNING TO ASSESS GAME BALANCING FOR COLLECTABLE CARD GAMES
This dissertation aims to create a framework for game balancing with suggestion while effectively exploring state space and non-deterministic information. It proposes a framework to assess and bring balance to Collectable Card Games automatically. Collectable Card Games have a complexity rule and provide non-deterministic information. In addition, Collectable Card Games also face the challenge of creating new cards that can integrate into the existing card pool without breaking the balance.

The game-balanced framework consists of three layers. The Player Layer, which includes self-play intelligence bots, aim to win this game. The Tournament Layer conducts multiple parallel games between the player’s self-play intelligence bots from the player layer and generates data through self-play games. The Game Balance Layer measures the predicted winning probability of each player, and the Game Balanced Layer demonstrates game re-balancing using machine learning, predictions and reasoning and then applies a new card to either player to generate perturbation into the game.
 
This framework aims to enable efficient identification and suggestion of any numerical parameters that cause imbalance and iterates over game parameters to restore game balance. This framework was designed and tested for a particular game. However, the background methodology can also apply to games such as Magic: The Gathering, Hearthstone, Pokémon Trading Card Game, and Yu-Gi-Oh! Trading Card Gam.
nc

## How to run this demonstration
You can run code in each cell in order from top including:

+ Section 1: Game Class
+ Section 2: Player Layer Class and Demonstrate agent training and playing between two layers.
+ Section 3: Tournament Layer Class
+ Section 4: Game Balance Class
## Introductory
These libraries need to be install before usage.


In [ ]:
pip install pandas
pip install numpy
pip install matplotlib
pip install ipywidgets
pip install pillow
pip install mcts-simple
pip install scikit-learn
pip install scikit-image
pip install shap

## Section 1: Game Class

This game has been implemented as a experiment for assess game balanced including player and board

In [ ]:
# Player and Card
import sys
import math
import os
import random
import csv
import statistics
import datetime
from PIL import Image

import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import savefig
from PIL import Image


class Player:
    def __init__(self, name, cardPath, startHp=15, startMp=4, maxMp=10, startCard=4):
        # Name
        # Position (Left/Right)
        self.name = name
        # Read Card from CSV
        self.df = pd.read_csv(os.getcwd() + "\\" + cardPath)
        # Player Stats
        self.mp = startMp
        self.maxMp = maxMp
        self.hp = [startHp,startHp,startHp]
        self.ctrl = [2,2,2]
        # Organise Deck
        self.drawStat = [0] * len(self.df.index)
        self.playStat = [0] * len(self.df.index)
        self.hand = []
        self.deck = [i for i in range(len(self.df.index))]
        self.board = []
        self.acted = False
        self.swaped = False
        self.drawCard(startCard)
        return

    def reset():
        # Player Stats
        self.mp = startMp
        self.maxMp = maxMp
        self.hp = [startHp,startHp,startHp]
        self.ctrl = [2,2,2]
        # Organise Deck
        self.drawStat = [0] * len(self.df.index)
        self.playStat = [0] * len(self.df.index)
        self.hand = []
        self.deck = [i for i in range(len(self.df.index))]
        self.board = []
        self.acted = False
        self.drawCard(startCard)

    def setMana(self, mana):
        self.mp = statistics.median([0,mana,self.maxMp])

    def addMana(self, mana):
        self.mp = statistics.median([0,self.mp + mana,self.maxMp])

    def drawCard(self, n):
        # Check if n more than deck
        n = len(self.deck) if n > len(self.deck) else n
        # Draw n cards from deck into hands.
        for i in range(n):
            pickedCard = random.choice(self.deck)
            self.hand.append(pickedCard)
            self.deck.remove(pickedCard)
            self.drawStat[pickedCard] = self.drawStat[pickedCard] + 1
        return

    def getCardInDeck(self):
        return self.df.loc[self.hand]

    def getCardInHand(self):
        return self.df.loc[self.hand]

    def printCardInHand(self, textMode=False, showAbility=False):
        print("Player's Card in Hand:")
        if(showAbility == True):
            if(textMode == True):
                return print(self.df.loc[self.hand])
            else:
                return self.df.loc[self.hand]
        else:
            if(textMode == True):
                return print(self.df.loc[self.hand,["cardName","mana","health","attack","desc"]])
            else:
                return self.df.loc[self.hand,["cardName","mana","health","attack","desc"]]
    

    def getCard(self, cardIndex):
        return self.df.loc[cardIndex]

    def usedCard(self, cardIndex):
        # Check if i card is in hand
        if(cardIndex in self.hand):
            # Append card list in board, remove from hand
            self.board.append(cardIndex)
            self.hand.remove(cardIndex)
            self.playStat[cardIndex] = self.playStat[cardIndex] + 1
            return
        else:
            raise Exception("Card Index Invalid")       

    def returnCard(self, cardIndex):
        # Check if i card is in hand
        if(cardIndex in self.hand):
            # Append card list in deck, remove from hand
            self.deck.append(cardIndex)
            self.hand.remove(cardIndex)
            return
        else:
            raise Exception("Card Index Invalid")

    def recallCard(self, cardIndex):
        # Check if i card is in hand
        if(cardIndex in self.board):
            # Append card list in deck, remove from board
            self.deck.append(cardIndex)
            self.board.remove(cardIndex)
        else:
            raise Exception("Card Index Invalid")
        return

class Board:
    ## self.WIDTH of Board
    WIDTH = 8

    def __init__(self, p1, p2, turnLimit=50):
        # Game & Frame
        self.frame = 0
        self.firstPlayer = 0
        # Turn Limit
        self.turn = 1
        self.turnLimit = turnLimit
        # Space & Tiles
        self.tiles = []
        self.spaces = []
        self.moves = []
        # Players
        self.p1 = p1
        self.p2 = p2
        self.gameTime = time.time()
    
    # Show avaliable action
    def getAvaliableAction(self, player, board):   
        avaliableAction = []
        # For each card in hand
        for cardIndex in player.hand:
            obj = player.getCard(cardIndex)
            # Check if mana is sufficent
            if(player.mp >= obj.mana):
                # Check all avaliable spaces
                for lane in range(3):
                    for dist in range(int(player.ctrl[lane])):
                        # Change distance in to col
                        if(player.name == self.p1.name):
                            col = dist
                        if(player.name == self.p2.name):
                            col = self.WIDTH-1-dist
                        # Check if location free to place
                        if(dist == 0 and player.hp[lane] == 0):
                            ## Do nothing, wall is destroyed
                            pass
                        elif(self.getBot(player.name, [lane, col]) != False):
                            ## Do nothing, location is not free
                            pass
                        else:
                            avaliableAction.append(tuple([cardIndex,lane,col]))
            else:
                pass
                ## Do nothing Mana not enough

        if(player.swaped == False):
            for cardIndex in player.hand:
                avaliableAction.append(tuple([cardIndex,-1,-1]))

        avaliableAction.append(tuple([-1,-1,-1]))
        random.shuffle(avaliableAction)
        return avaliableAction
        
    # Place Bot
    def placeBot(self, player, cardIndex, location):
        obj = player.getCard(cardIndex)
        dict = {"Player": player.name,
                "CardIndex": cardIndex,
                "Name": obj.cardName,
                "Health": obj.health,
                "Attack": obj.attack,
                "AbilityName": obj.ability,
                "AbilityParam": int(obj.param),
                "AbilityCondition": obj.condition,
                "AbilityTrigger": obj.trigger,
                "Location": (location[0],location[1]),
                "SpecialEffect": "NaN"
               }
        self.spaces.append(dict)
        self.moves.append(dict)
        player.addMana(-obj.mana)
        #print(player.name + " Mana Left: " + str(player.mp))
        player.usedCard(cardIndex)
        return

    def discardCard(self, player, cardIndex):
        player.returnCard(cardIndex)
        player.drawCard(1)
        player.swaped = True
        return

    # Proceed - All action finished, move a bot, inflict tile ailment and attack
    def proceed(self):
        # Effect Start
        for bot in self.spaces:
            if(bot["AbilityTrigger"] == "StartOnce" or bot["AbilityTrigger"] == "Start"):
                if(self.isEffectMetCondition(bot) == True):
                    # AllStatsUp - Increase both HP and Attack
                    if(bot["AbilityName"] == "AllStatsUp"):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " HP rise from " + str(bot["Health"]) + " to " + str(bot["Health"] + bot["AbilityParam"]))
                        bot["Health"] = bot["Health"] + bot["AbilityParam"]
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " ATK rise from " + str(bot["Attack"]) + " to " + str(bot["Attack"] + bot["AbilityParam"]))
                        bot["Attack"] = bot["Attack"] + bot["AbilityParam"]
                    # PlaceFireTrap - Place fire trap
                    elif(bot["AbilityName"] == "PlaceFireTrap"):
                        dict = {"Player": bot["Player"],
                                "Name": "Fire",
                                "Param": bot["AbilityParam"],
                                "Location": bot["Location"]
                               }
                        self.tiles.append(dict)
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " Fire Trap Set")
                    # AtkUpAdjacent - Increase Attack on bot on the left and right
                    elif(bot["AbilityName"] == "AtkUpAdjacent"):
                        if(bot["Location"][0] == 0 or bot["Location"][0] == 2):
                            if(self.getBot(bot["Player"],[1,bot["Location"][1]]) != False and self.getBot(bot["Player"],[1,bot["Location"][1]])["Player"] == bot["Player"]):
                                botAdj = self.getBot(bot["Player"],[1,bot["Location"][1]])
                                #print(botAdj["Player"] + "'s " + botAdj["Name"] + " @ " + str(botAdj["Location"][0]) + "," + str(botAdj["Location"][1])  + " ATK rise from " + str(botAdj["Attack"]) + " to " + str(botAdj["Attack"] + bot["AbilityParam"]))
                                botAdj["Attack"] = botAdj["Attack"] + bot["AbilityParam"]
                        elif(bot["Location"][0] == 1):
                            if(self.getBot(bot["Player"],[0,bot["Location"][1]]) != False and self.getBot(bot["Player"],[0,bot["Location"][1]])["Player"] == bot["Player"]):
                                botAdj = self.getBot(bot["Player"],[0,bot["Location"][1]])
                                #print(botAdj["Player"] + "'s " + botAdj["Name"] + " @ " + str(botAdj["Location"][0]) + "," + str(botAdj["Location"][1])  + " ATK rise from " + str(botAdj["Attack"]) + " to " + str(botAdj["Attack"] + bot["AbilityParam"]))
                                botAdj["Attack"] = botAdj["Attack"] + bot["AbilityParam"]
                            if(self.getBot(bot["Player"],[2,bot["Location"][1]]) != False and self.getBot(bot["Player"],[2,bot["Location"][1]])["Player"]):
                                botAdj = self.getBot(bot["Player"],[2,bot["Location"][1]])
                                #print(botAdj["Player"] + "'s " + botAdj["Name"] + " @ " + str(botAdj["Location"][0]) + "," + str(botAdj["Location"][1])  + " ATK rise from " + str(botAdj["Attack"]) + " to " + str(botAdj["Attack"] + bot["AbilityParam"]))
                                botAdj["Attack"] = botAdj["Attack"] + bot["AbilityParam"]
                    # Shockwave - Increase Attack on bot on the left and right
                    elif(bot["AbilityName"] == "Shockwave"):
                        if(bot["Player"] == self.p1.name):
                            if(self.getFrontestBotInLane(self.p2.name,bot["Location"][0]) != False):
                                botAim = self.getFrontestBotInLane(self.p2.name,bot["Location"][0])
                                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal shockwave " + str(bot["AbilityParam"]) + " damage to " + botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1]) )
                                #print(botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1])  + " HP fall from " + str(botAim["Health"]) + " to " + str(botAim["Health"] - bot["AbilityParam"]))
                                botAim["Health"] = botAim["Health"] - bot["AbilityParam"]
                            else:
                                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal shockwave " + str(bot["AbilityParam"]) + " opponent's wall in lane " + str(bot["Location"][0]))
                                #print(self.p2.name + "'s wall in lane " + str(bot["Location"][0]) + " HP fall from " + str(self.p2.hp[bot["Location"][0]]) + " to " + str(self.p2.hp[bot["Location"][0]] - bot["AbilityParam"]))
                                self.p2.hp[bot["Location"][0]] = max([0, self.p2.hp[bot["Location"][0]] - bot["AbilityParam"]])
                        elif(bot["Player"] == self.p2.name):
                            if(self.getFrontestBotInLane(self.p1.name,bot["Location"][0]) != False):
                                botAim = self.getFrontestBotInLane(self.p1.name,bot["Location"][0])
                                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal shockwave " + str(bot["AbilityParam"]) + " damage to " + botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1]) )
                                #print(botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1])  + " HP fall from " + str(botAim["Health"]) + " to " + str(botAim["Health"] - bot["AbilityParam"]))
                                botAim["Health"] = botAim["Health"] - bot["AbilityParam"]
                            else:
                                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal shockwave " + str(bot["AbilityParam"]) + " opponent's wall in lane " + str(bot["Location"][0]))
                                #print(self.p1.name + "'s wall in lane "+ str(bot["Location"][0]) + " HP fall from " + str(self.p1.hp[bot["Location"][0]]) + "to " + str(self.p1.hp[bot["Location"][0]] - bot["AbilityParam"]))
                                self.p1.hp[bot["Location"][0]] = max([0, self.p1.hp[bot["Location"][0]] - bot["AbilityParam"]])
                    # ManaBonus - Add Mana at the start of the game
                    elif(bot["AbilityName"] == "ManaBonus"):
                        if(bot["Player"] == self.p1.name):
                            self.p1.addMana(bot["AbilityParam"])
                            #print(self.p1.name + " gain bonus " + str(bot["AbilityParam"]) + " mana (Total=" + str(self.p1.mp) + ")")
                        elif(bot["Player"] == self.p2.name):
                            self.p2.addMana(bot["AbilityParam"])
                            #print(self.p2.name + " gain bonus " + str(bot["AbilityParam"]) + " mana (Total=" + str(self.p2.mp) + ")")
                    # Place Energy Tile
                    elif(bot["AbilityName"] == "EnergyTile"):
                        dict = {"Player": bot["Player"],
                                "Name": "Energy",
                                "Param": bot["AbilityParam"],
                                "Location": bot["Location"]
                               }
                        self.tiles.append(dict)
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " Energy Tile Set")
                    # Allies Atk Up
                    elif(bot["AbilityName"] == "AlliesAtkUp"):
                        for botAim in self.spaces:
                            if(botAim["Player"] == bot["Player"] and botAim != bot):
                                #print(botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1])  + " Attack rise from " + str(botAim["Attack"]) + " to " + str(botAim["Attack"] + bot["AbilityParam"]))
                                botAim["Attack"] = botAim["Attack"] + bot["AbilityParam"]
                    # Atk Up
                    elif(bot["AbilityName"] == "AtkUp"):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " ATK rise from " + str(bot["Attack"]) + " to " + str(bot["Attack"] + bot["AbilityParam"]))
                        bot["Attack"] = bot["Attack"] + bot["AbilityParam"]
                    # Place Frozen Tile
                    elif(bot["AbilityName"] == "FrozenTile"):
                        dict = {"Player": bot["Player"],
                                "Name": "Frozen",
                                "Param": bot["AbilityParam"],
                                "Location": bot["Location"]
                               }
                        self.tiles.append(dict)
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " Ice Trap Set")
            # If Start Once, change to used status
            if(bot["AbilityTrigger"] == "StartOnce"):
                bot["AbilityTrigger"] = "StartUsed"
                
        # March Forward
        for bot in self.spaces:
            # If there is a wall in front, cannot move
            if((bot["Player"] == self.p1.name and bot["Location"][1] == self.WIDTH-2) or (bot["Player"] == self.p2.name and bot["Location"][1] == 1)):
                pass
                #print("Cannot Move " + bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " because this bot is in front of opponent's wall.")
            # If there is a bot in front, check if bot can jump
            elif(self.getBot(bot["Player"],bot["Location"], 1) != False):
                # If Jump, Check 2 spaces
                if(bot["AbilityName"] == "Jump" and self.getBot(bot["Player"],bot["Location"], 2) == False and bot["Location"][1]+2 <= self.WIDTH-1 and bot["Location"][1]-2 >= 1):
                    if(bot["Player"] == self.p1.name):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " jump to " + str(bot["Location"][0]) + "," + str(bot["Location"][1]+2))
                        bot["Location"] = [bot["Location"][0],bot["Location"][1]+2]
                    elif(bot["Player"] == self.p2.name):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " jump to " + str(bot["Location"][0]) + "," + str(bot["Location"][1]-2))
                        bot["Location"] = [bot["Location"][0],bot["Location"][1]-2]
                else:
                    pass
                    #print("Cannot Move " + bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " because there is obstacle in front of them.")
            # If there is a bot two space in a front, only bot that near net (self.WIDTH/2 + 0.5) will move
            elif(self.getBot(bot["Player"],bot["Location"], 2) != False and self.spaces.index(self.getBot(bot["Player"],bot["Location"], 2)) > self.spaces.index(bot)):
                if(bot["Player"] == self.p1.name and abs(bot["Location"][1] - self.WIDTH/2.0) < abs(bot["Location"][1]+2 - self.WIDTH/2.0)):
                    #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " move to " + str(bot["Location"][0]) + "," + str(bot["Location"][1]+1) + " (occupied via net)")
                    bot["Location"] = [bot["Location"][0],bot["Location"][1]+1]
                elif(bot["Player"] == self.p2.name and abs(bot["Location"][1] - self.WIDTH/2.0) < abs(bot["Location"][1]-2 - self.WIDTH/2.0)):
                    #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " move to " + str(bot["Location"][0]) + "," + str(bot["Location"][1]-1) + " (occupied via net)")
                    bot["Location"] = [bot["Location"][0],bot["Location"][1]-1]
                else:
                    pass
                    #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " cannot move because there you are far from net more than opponent.")
            # If it has frozen tile, frozen for one turn
            elif(self.isTileApplied(bot,"Frozen") > 0):
                if(bot["SpecialEffect"] == "Freeze"):
                    if(bot["Player"] == self.p1.name):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " move to " + str(bot["Location"][0]) + "," + str(bot["Location"][1]+1))
                        bot["Location"] = [bot["Location"][0],bot["Location"][1]+1]
                    elif(bot["Player"] == self.p2.name):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " move to " + str(bot["Location"][0]) + "," + str(bot["Location"][1]-1))
                        bot["Location"] = [bot["Location"][0],bot["Location"][1]-1]
                    bot["SpecialEffect"] = "NaN"
                else:
                    #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " is frozen!")
                    bot["SpecialEffect"] == "Freeze"
            # else, moving
            else:
                if(bot["Player"] == self.p1.name):
                    #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " move to " + str(bot["Location"][0]) + "," + str(bot["Location"][1]+1))
                    bot["Location"] = [bot["Location"][0],bot["Location"][1]+1]
                elif(bot["Player"] == self.p2.name):
                    #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " move to " + str(bot["Location"][0]) + "," + str(bot["Location"][1]-1))
                    bot["Location"] = [bot["Location"][0],bot["Location"][1]-1]

        # Effect Tile
        for bot in self.spaces:
            if(self.isTileApplied(bot,"Fire") > 0):
                #print(#printBot(bot["Player"], bot["Name"], bot["Location"]) + "is burnt")
                #print(#printBot(bot["Player"], bot["Name"], bot["Location"]) + " HP reduce from " + str(bot["Health"]) + " to " + str(bot["Health"] - self.isTileApplied(bot,"Fire")))
                bot["Health"] -= self.isTileApplied(bot,"Fire")

        # Attack Bot
        for bot in self.spaces:
            # Get Atk
            atk = bot["Attack"]
            if(bot["AbilityTrigger"] == "Attack"):
                if(self.isEffectMetCondition(bot)):
                    if(bot["AbilityName"] == "AtkUp"):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " ATK rise from " + str(bot["Attack"]) + " to " + str(bot["Attack"] + bot["AbilityParam"]))
                        atk = atk + bot["AbilityParam"]
                        
            if(self.isTileApplied(bot,"Energy") > 0):
                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + "is energised!")
                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " ATK rise from " + str(bot["Attack"]) + " to " + str(bot["Attack"] + self.isTileApplied(bot,"Energy")))
                atk = atk + self.isTileApplied(bot,"Energy")
            
            ### Get aim bot
            if(bot["AbilityTrigger"] == "Attack" and (bot["AbilityName"] == "DiagAttack" or bot["AbilityName"] == "WideAttack")):
                if(self.isEffectMetCondition(bot)):
                    # Diag attack
                    if(bot["AbilityName"] == "DiagAttack"):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " perform diagonal attack")
                        if(bot["Location"][0] == 0 or bot["Location"][0] == 2):
                            if(self.getBot(bot["Player"],[1,bot["Location"][1]], 1) != False and self.getBot(bot["Player"],[1,bot["Location"][1]], 1)["Player"] != bot["Player"]):
                                opponentBot = self.getBot([1,bot["Location"][1]], 1)
                                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " ATK rise from " + str(bot["Attack"]) + " deal " + str(bot["Attack"]) + " damage to " + opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1]) )
                                # Check defense
                                if(opponentBot != False and opponentBot["Player"] != bot["Player"] and opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                                    #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                                    opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                                elif(opponentBot != False and opponentBot["Player"] != bot["Player"]):
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                                    opponentBot["Health"] -= atk
                        elif(bot["Location"][0] == 1):
                            if(self.getBot(bot["Player"],[0,bot["Location"][1]], 1) != False and self.getBot(bot["Player"],[0,bot["Location"][1]], 1)["Player"] != bot["Player"]):
                                opponentBot = self.getBot(bot["Player"],[0,bot["Location"][1]], 1)
                                # Check defense
                                if(opponentBot != False and opponentBot["Player"] != bot["Player"] and opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                                    #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                                    opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                                elif(opponentBot != False and opponentBot["Player"] != bot["Player"]):
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                                    opponentBot["Health"] -= atk
                            if(self.getBot(bot["Player"],[2,bot["Location"][1]], 1) != False and self.getBot(bot["Player"],[2,bot["Location"][1]], 1)["Player"] != bot["Player"]):
                                opponentBot = self.getBot(bot["Player"],[2,bot["Location"][1]], 1)
                                # Check defense
                                if(opponentBot != False and opponentBot["Player"] != bot["Player"] and opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                                    #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                                    opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                                elif(opponentBot != False and opponentBot["Player"] != bot["Player"]):
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                                    opponentBot["Health"] -= atk
                    # Wide attack
                    elif(bot["AbilityName"] == "WideAttack"):
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " perform wide attack")
                        if(bot["Location"][0] == 0 or bot["Location"][0] == 1):
                            if(self.getBot(bot["Player"],[0,bot["Location"][1]], 1) != False and self.getBot(bot["Player"],[0,bot["Location"][1]], 1) != bot["Player"]):
                                opponentBot = self.getBot(bot["Player"],[0,bot["Location"][1]], 1)
                                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal " + str(bot["Attack"]) + " damage to " + opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1]) )
                                # Check defense
                                if(opponentBot != False and opponentBot["Player"] != bot["Player"] and opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                                    #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                                    opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                                elif(opponentBot != False and opponentBot["Player"] != bot["Player"]):
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                                    opponentBot["Health"] -= atk
                        if(bot["Location"][0] == 0 or bot["Location"][0] == 1 or bot["Location"][0] == 2):
                            if(self.getBot(bot["Player"],[1,bot["Location"][1]], 1) != False and self.getBot(bot["Player"],[1,bot["Location"][1]], 1) != bot["Player"]):
                                opponentBot = self.getBot(bot["Player"],[1,bot["Location"][1]], 1)
                                #print(self.#printBot(bot["Player"], bot["Name"], bot["Location"]) + " deal " + str(bot["Attack"]) + " damage to " + self.#printBot(opponentBot["Player"], opponentBot["Name"], opponentBot["Location"]))
                                # Check defense
                                if(opponentBot != False and opponentBot["Player"] != bot["Player"] and opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                                    #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                                    opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                                elif(opponentBot != False and opponentBot["Player"] != bot["Player"]):
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                                    opponentBot["Health"] -= atk
                        if(bot["Location"][0] == 1 or bot["Location"][0] == 2):
                            if(self.getBot(bot["Player"],[2,bot["Location"][1]], 1) != False and self.getBot(bot["Player"],[2,bot["Location"][1]], 1) != bot["Player"]):
                                opponentBot = self.getBot(bot["Player"], [2,bot["Location"][1]], 1)
                                #print(self.#printBot(bot["Player"], bot["Name"], bot["Location"]) + " deal " + str(bot["Attack"]) + " damage to " + self.#printBot(opponentBot["Player"], opponentBot["Name"], opponentBot["Location"]))
                                # Check defense
                                if(opponentBot != False and opponentBot["Player"] != bot["Player"] and opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                                    #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                                    opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                                elif(opponentBot != False and opponentBot["Player"] != bot["Player"]):
                                    #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                                    opponentBot["Health"] -= atk
            #else
            else:                        
                # if it is not special attack, perform normal attack
                if(self.getBot(bot["Player"],bot["Location"], 1) != False and self.getBot(bot["Player"],bot["Location"], 1) != bot["Player"]):
                    opponentBot = self.getBot(bot["Player"],bot["Location"], 1)
                    ## Frozen
                    if(opponentBot != False and opponentBot["Player"] != bot["Player"] and opponentBot["SpecialEffect"] == "Freeze" and bot["AbilityCondition"] == "EnemyFrozenBot"):
                        atk = atk + bot["AbilityParam"]
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " gain extra " +  bot["AbilityParam"] + " attack from frozen target")     
                        #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal " + str(bot["Attack"]) + " damage to " + opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1]) )                    
                        # Check defense
                        if(opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                            #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                            #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                            opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                        else:
                            #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                            opponentBot["Health"] -= atk
                    elif(opponentBot != False and opponentBot["Player"] != bot["Player"]):
                        if(opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                            #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                            #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                            opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                        else:
                            #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                            opponentBot["Health"] -= atk
                    # If bot can shoot from 2 range
                    elif(opponentBot == False and bot["AbilityTrigger"] == "Attack" and bot["AbilityName"] == "Range"):
                        if(self.getBot(bot["Player"],bot["Location"], 2) != False and self.getBot(bot["Player"],bot["Location"], 2) != bot["Player"]):
                            opponentBot = self.getBot(bot["Player"],bot["Location"], 2)
                            #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal " + str(bot["Attack"]) + " damage to " + opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1]) )                    
                            # Check defense
                            if(opponentBot["AbilityName"] == "Defense" and self.isEffectMetCondition(opponentBot)):
                                #print("Opponent Guard " + opponentBot["AbilityParam"] + "dmg")
                                #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - max(0,atk-opponentBot["AbilityParam"])))
                                opponentBot["Health"] -= max(0,atk-opponentBot["AbilityParam"])
                            else:
                                #print(opponentBot["Player"] + "'s " + opponentBot["Name"] + " @ " + str(opponentBot["Location"][0]) + "," + str(opponentBot["Location"][1])  + " HP fall from " + str(opponentBot["Health"]) + " to " + str(opponentBot["Health"] - atk))
                                opponentBot["Health"] -= atk

        # Attack Tower
        for bot in self.spaces:
            if(bot["Player"] == self.p1.name and bot["Location"][1] == self.WIDTH-2 and self.getBot(bot["Player"],bot["Location"], 1) == False and self.p2.hp[bot["Location"][0]] > 0):
                prev_hp = self.p2.hp[bot["Location"][0]]
                self.p2.hp[bot["Location"][0]] = max([self.p2.hp[bot["Location"][0]] - bot["Attack"], 0])
                #print(self.p2.name + "'s Wall in lane " + str(bot["Location"][0]) + " HP fall from " + str(prev_hp) + " to " + str(self.p2.hp[bot["Location"][0]]))
            elif(bot["Player"] == self.p2.name and bot["Location"][1] == 1 and self.getBot(bot["Player"],bot["Location"], 1) == False and self.p1.hp[bot["Location"][0]] > 0):
                prev_hp = self.p1.hp[bot["Location"][0]]
                self.p1.hp[bot["Location"][0]] = max([self.p1.hp[bot["Location"][0]] - bot["Attack"], 0])
                #print(self.p1.name + "'s Wall in lane " + str(bot["Location"][0]) + " HP fall from " + str(prev_hp) + " to " + str(self.p1.hp[bot["Location"][0]]))

        mana_bonus_p1 = 0
        mana_bonus_p2 = 0
        # Mana Bonus, Recall cards
        for bot in self.spaces:
            # If there is a bot in front, attack
            if(bot["Health"] <= 0):
                # Ability
                if(bot["AbilityTrigger"] == "Wipe"):
                    if(self.isEffectMetCondition(bot)):
                        if(bot["AbilityName"] == "Bomb"):
                            if(bot["Player"] == self.p1.name and self.getBot(bot["Player"],bot["Location"],1) != False and self.getBot(bot["Player"],bot["Location"],1) != False and self.getBot(bot["Player"],bot["Location"],1)["Player"] == self.p2.name):
                                botAim = self.getBot(bot["Player"],bot["Location"],1)
                                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal bomb " + str(bot["AbilityParam"]) + " damage to " + botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1]))
                                #print(botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1]) + " Health reduce from " + str(botAim["Health"]) + " to " + (botAim["Health"] - bot["AbilityParam"]))
                                botAim["Health"] = botAim["Health"] - bot["AbilityParam"]
                            if(bot["Player"] == self.p2.name and self.getBot(bot["Player"],bot["Location"],1) != False and self.getBot(bot["Player"],bot["Location"],1) != False and self.getBot(bot["Player"],bot["Location"],1)["Player"] == self.p1.name):
                                botAim = self.getBot(bot["Player"],bot["Location"],1)
                                #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " deal bomb " + str(bot["AbilityParam"]) + " damage to " + botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1]))
                                #print(botAim["Player"] + "'s " + botAim["Name"] + " @ " + str(botAim["Location"][0]) + "," + str(botAim["Location"][1]) + " Health reduce from " + str(botAim["Health"]) + " to " + (botAim["Health"] - bot["AbilityParam"]))
                                botAim["Health"] = botAim["Health"] - bot["AbilityParam"]

        for bot in self.spaces:
            # If there is a bot in front, attack
            if(bot["Health"] <= 0):
                if(bot["Player"] == self.p1.name):
                    #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " removed")
                    mana_bonus_p2 = mana_bonus_p2 + 1
                    self.p1.recallCard(bot["CardIndex"])
                elif(bot["Player"] == self.p2.name):
                    #print(bot["Player"] + "'s " + bot["Name"] + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + " removed")
                    mana_bonus_p1 = mana_bonus_p1 + 1
                    self.p2.recallCard(bot["CardIndex"])

        # Wipe Bot
        for bot in self.spaces:
            if(bot["AbilityTrigger"] == "Wipe"):
                if(self.isEffectMetCondition(bot)):
                    if(bot["AbilityCondition"] == "OwnBotWipe" and ((bot["Player"] == self.p1.name and mana_bonus_p1 > 0) or (bot["Player"] == self.p2.name and mana_bonus_p2 > 0))):
                            if(bot["AbilityName"] == "AllStatUp"):
                                #print(self.#printBot(bot["Player"], bot["Name"], bot["Location"]) + " Health rise from " + str(bot["Health"]) + " to " + str(bot["Health"] + bot["AbilityParam"]))
                                bot["Health"] = bot["Health"] + bot["AbilityParam"]
                                #print(self.#printBot(bot["Player"], bot["Name"], bot["Location"]) + " Attack rise from " + str(bot["Attack"]) + " to " + str(bot["Attack"] + bot["AbilityParam"]))
                                bot["Attack"] = bot["Attack"] + bot["AbilityParam"]
    
        #check if bot reach opposite side of wall 
        for lane in range(3):
            bot = self.getBot("def",[lane,1])
            if((bot != False) and (bot["Player"] == self.p2.name) and (self.p1.hp[lane] == 0)):
                if((bot["CardIndex"] in self.p2.board)):
                    self.p2.recallCard(bot["CardIndex"])
            bot = self.getBot("def",[lane,self.WIDTH-2])
            if((bot != False) and (bot["Player"] == self.p1.name) and (self.p2.hp[lane] == 0)):
                if((bot["CardIndex"] in self.p1.board)):
                    self.p1.recallCard(bot["CardIndex"])
                    
        ## Remove from list              
        self.spaces = [bot for bot in self.spaces if bot["Health"] > 0]
        for lane in range(3):
            self.spaces = [bot for bot in self.spaces if not(bot["Location"] == [lane,1] and bot["Player"] == self.p2.name and self.p1.hp[lane] == 0)]
            self.spaces = [bot for bot in self.spaces if not(bot["Location"] == [lane,self.WIDTH-2] and bot["Player"] == self.p1.name and self.p2.hp[lane] == 0)]
            
        ## Add Mana Bonus
        #print("Player 1 eliminated " + str(mana_bonus_p1) + " bot(s). Gain " + str(mana_bonus_p1) + " mana.")
        self.p1.addMana(mana_bonus_p1)
        #print("Player 2 eliminated " + str(mana_bonus_p2) + " bot(s). Gain " + str(mana_bonus_p2) + " mana.")
        self.p2.addMana(mana_bonus_p2)

        self.p1.addMana(2)
        self.p2.addMana(2)
        
        # Distribute Mana
        if(self.p1.hp.count(0) >= 1):
            self.p1.addMana(2)
            #print(self.p1.name + " gain 1 additional mana from a wall destroyed")
        if(self.p2.hp.count(0) >= 1):
            self.p2.addMana(2)
            #print(self.p2.name + " gain 1 additional mana from a wall destroyed")
        
        #print(self.p1.name + " mana = " + str(self.p1.mp))
        #print(self.p2.name + " mana = " + str(self.p2.mp))

        # Draw Card
        if(len(self.p1.hand) < 4):
            self.p1.drawCard(1)
        elif(len(self.p2.hand) < 4):
            self.p2.drawCard(1)

        # Expand Turf
        for i in range(3):
            if(self.getFrontestBotInLane(self.p1.name,i) != False):
                self.p1.ctrl[i] = statistics.median([int(2), self.getFrontestBotInLane(self.p1.name,i)["Location"][1] + 1, int(self.WIDTH/2)])
            else:
                self.p1.ctrl[i] = 2
        for i in range(3):
            if(self.getFrontestBotInLane(self.p2.name,i) != False):
                self.p2.ctrl[i] = statistics.median([int(2), self.WIDTH - self.getFrontestBotInLane(self.p2.name,i)["Location"][1], int(self.WIDTH/2)])
            else:
                self.p2.ctrl[i] = 2
                
        self.p1.acted = False
        self.p2.acted = False
        self.p1.swaped = False
        self.p2.swaped = False
        self.turn = self.turn + 1 
        return

    #reset
    def reset(self):
        self.turn = 1
        self.turnLimit = turnLimit
        # Space & Tiles
        self.tiles = []
        self.spaces = []
        self.moves = []
        # Players
        self.p1.reset()
        self.p2.reset()

    ## Check if effect it is right condition
    def isEffectMetCondition(self, bot):
        if(bot["AbilityCondition"] == "PlaceTower"):
            if((bot["Player"] == self.p1.name and bot["Location"][1] == 0) or (bot["Player"] == self.p2.name and bot["Location"][1] == self.WIDTH-1)):
                return True
            else:
                return False
        elif(bot["AbilityCondition"] == "Death"):
            if(bot["Health"] <= 0):
                return True
            else:
                return False
        elif(bot["AbilityCondition"] == "2InLane"):
            if(self.getNumberOfBot([bot["Player"]], [bot["Location"][0]]) >= 2):
                return True
            else:
                return False
        elif(bot["AbilityCondition"] == "IsInFront"):
            if(self.getOrderOfBotInLane(bot["Player"],bot["Location"]) == 0):
                return True
            else:
                return False
        elif(bot["AbilityCondition"] == "EnemyTurf"):
            if((bot["Player"] == self.p1.name and bot["Location"][1] > (self.WIDTH/2)) or (bot["Player"] == self.p2.name and bot["Location"][1] < (self.WIDTH/2))):
                return True
            else:
                return False
        return True

    def getState(self):
        list = []
        # Turn
        list.append(self.turn)
        # P1 Wall HP
        list.append(self.p1.hp[0])
        list.append(self.p1.hp[1])
        list.append(self.p1.hp[2])
        # P2 Wall HP
        list.append(self.p2.hp[0])
        list.append(self.p2.hp[1])
        list.append(self.p2.hp[2])
        # P1 MP
        list.append(self.p1.mp)
        # P2 MP
        list.append(self.p2.mp)
        # Board (Health)
        for lane in range(3):
            for col in range(self.WIDTH):
                if(self.getBot("def",[lane,col]) != False):
                    if(self.getBot("def",[lane,col])["Player"] == self.p1.name):
                        list.append(self.getBot("def",[lane,col])["Health"])
                    elif(self.getBot("def",[lane,col])["Player"] == self.p2.name):
                        list.append(-self.getBot("def",[lane,col])["Health"])
                else:
                    list.append(0)
        # Board (Attack)
        for lane in range(3):
            for col in range(self.WIDTH):
                if(self.getBot("def",[lane,col]) != False):
                    if(self.getBot("def",[lane,col])["Player"] == self.p1.name):
                        list.append(self.getBot("def",[lane,col])["Attack"])
                    elif(self.getBot("def",[lane,col])["Player"] == self.p2.name):
                        list.append(-self.getBot("def",[lane,col])["Attack"])
                else:
                    list.append(0)
        return list

    def getStateCNN(self):
        list_r = []
        list_g = []
        list_b = []
        # HP P1 Lane 0, 1, 2 (8x6)
        for lane in range(3):
            for gauge in range(16):
                if(self.p1.hp[lane] > gauge + 1):
                    list_r.append(255)
                    list_g.append(0)
                    list_b.append(0)
                else:
                    list_r.append(0)
                    list_g.append(0)
                    list_b.append(0)
        #print(len(list))
        # MP P1 (8x2)
        for gauge in range(16):
            if(self.p1.mp > gauge + 1):
                list_r.append(255)
                list_g.append(0)
                list_b.append(0)
            else:
                list_r.append(0)
                list_g.append(0)
                list_b.append(0)
        #print(len(list))
        # Card in Deck P1 (8x2)
        for cardIndex in range(16):
            if(cardIndex in self.p1.deck):
                list_r.append(10*cardIndex)
                list_g.append(0)
                list_b.append(0)
            else:
                list_r.append(0)
                list_g.append(0)
                list_b.append(0)
        #print(len(list))
        # Card in Hand P1 (8x2)
        for cardIndex in range(15):
            if(cardIndex in self.p1.hand):
                list_r.append(10*cardIndex)
                list_g.append(0)
                list_b.append(0)
            else:
                list_r.append(0)
                list_g.append(0)
                list_b.append(0)
        if(self.firstPlayer == 1):
            list_r.append(255)
            list_g.append(0)
            list_b.append(0)
        else:
            list_r.append(0)
            list_g.append(0)
            list_b.append(0)
        #print(len(list))
        # Board (8x3)
        for lane in range(3):
            for col in range(self.WIDTH):
                bot = self.getBot("def",[lane,col])
                if(bot != False):
                    if(bot["Player"] == self.p1.name):
                        list_r.append(10*bot["CardIndex"])
                        list_g.append(5*bot["Health"])
                        list_b.append(0)
                    elif(bot["Player"] == self.p2.name):
                        list_r.append(0)
                        list_g.append(5*bot["Health"])
                        list_b.append(10*bot["CardIndex"])
                    else:
                        list_r.append(0)
                        list_g.append(0)
                        list_b.append(0)
                else:
                    list_r.append(0)
                    list_g.append(0)
                    list_b.append(0)
        #print(len(list))
        # Card in Hand P2 (8x2)
        for cardIndex in range(15):
            if(cardIndex in self.p2.hand):
                list_r.append(0)
                list_g.append(0)
                list_b.append(10*cardIndex)
            else:
                list_r.append(0)
                list_g.append(0)
                list_b.append(0)
        if(self.firstPlayer == 0):
            list_r.append(0)
            list_g.append(0)
            list_b.append(255)
        else:
            list_r.append(0)
            list_g.append(0)
            list_b.append(0)
        #print(len(list))
        # Card in Deck P2 (8x2)
        for cardIndex in range(16):
            if(cardIndex in self.p2.deck):
                list_r.append(0)
                list_g.append(0)
                list_b.append(10*cardIndex)
            else:
                list_r.append(0)
                list_g.append(0)
                list_b.append(0)
        #print(len(list))
        # MP P2  (8x2)
        for gauge in range(16):
            if(self.p2.mp > gauge + 1):
                list_r.append(0)
                list_g.append(0)
                list_b.append(255)
            else:
                list_r.append(0)
                list_g.append(0)
                list_b.append(0)
        #print(len(list))
        # HP P2 Lane 2, 1, 0  (8x6)
        for lane in range(3):
            for gauge in range(16):
                if(self.p2.hp[2-lane] > gauge + 1):
                    list_r.append(0)
                    list_g.append(0)
                    list_b.append(255)
                else:
                    list_r.append(0)
                    list_g.append(0)
                    list_b.append(0)

        arr_r=np.array(list_r,dtype="uint8")
        arr_g=np.array(list_g,dtype="uint8")
        arr_b=np.array(list_b,dtype="uint8")

        reshaper=arr_r.reshape(27,8)
        reshapeg=arr_g.reshape(27,8)
        reshapeb=arr_b.reshape(27,8)
        
        imr=Image.fromarray(reshaper,mode=None) # mode I
        img=Image.fromarray(reshapeg,mode=None)
        imb=Image.fromarray(reshapeb,mode=None)
        
        merged=Image.merge("RGB",(imr,img,imb))
        #print(np_data)
        return merged

    def getStateTitleLOG(self):
        list = []
        list.append("TIME")
        list.append("TURN")
        list.append("P1_HP0")
        list.append("P1_HP1")
        list.append("P1_HP2")
        list.append("P1_MP")
        list.append("P1_BOARD")
        list.append("P1_HAND")
        list.append("P1_DECK")
        list.append("P2_HP0")
        list.append("P2_HP1")
        list.append("P2_HP2")
        list.append("P2_MP")
        list.append("P2_BOARD")
        list.append("P2_HAND")
        list.append("P2_DECK")
        return list

    def getStateLOG(self):
        list = []
        list.append(time.time())
        list.append(self.turn)
        list.append(self.p1.hp[0])
        list.append(self.p1.hp[1])
        list.append(self.p1.hp[2])
        list.append(self.p1.mp)
        list.append(len(self.p1.board))
        list.append(len(self.p1.hand))
        list.append(len(self.p1.deck))
        list.append(self.p2.hp[0])
        list.append(self.p2.hp[1])
        list.append(self.p2.hp[2])
        list.append(self.p2.mp)
        list.append(len(self.p2.board))
        list.append(len(self.p2.hand))
        list.append(len(self.p2.deck))
        return list
        
            
    def getStateTitleSHAP(self):
        list = []
        # P1 Card
        for index in range(14):
            list.append("P1_PLAY_" + str(index))
        # for index in range(14):
        #     list.append("P1_DRAW_" + str(index))
        # P2 Card
        for index in range(14):
            list.append("P2_PLAY_" + str(index))
        # for index in range(14):
        #     list.append("P2_DRAW_" + str(index))
        # Outcome
        list.append("P1WIN")
        list.append("P2WIN")
        return list

    def getStateSHAP(self):
        list = []
        # P1 Card
        for index in range(14):
            if(float(self.p1.drawStat[index]) > 0):
                list.append(float(self.p1.playStat[index])/float(self.p1.drawStat[index]))
            else:
                list.append(0.0)
        # for index in range(14):
        #     list.append(float(self.p1.drawStat[index]))
        # P2 Card
        for index in range(14):
            if(float(self.p2.drawStat[index]) > 0):
                list.append(float(self.p2.playStat[index])/float(self.p2.drawStat[index]))
            else:
                list.append(0.0)
        # for index in range(14):
        #     list.append(float(self.p2.drawStat[index]))
        # Outcome
        if(self.checkResult() == False):
            list.append(0)
            list.append(0)
        else:
            if(0 in self.checkResult() and 1 in self.checkResult()):
                list.append(0)
                list.append(0)
            elif(0 in self.checkResult()):
                list.append(1)       
                list.append(-1)
            elif(1 in self.checkResult()):
                list.append(-1)
                list.append(1)       
        return list

    # Check win condition
    def checkResult(self):
        ### if two wall perish win
        if(self.p1.hp.count(0) >= 2 and self.p2.hp.count(0) >= 2):
            ## compare by the median wall
            if(max(self.p1.hp) > max(self.p2.hp)):
                #print("The second lowest wall P1 is stronger than P2. Player 1 win! [P1-P2: " + str(max(wall_p1)) + "-" + str(max(wall_p2)) + "]")
                return [0]
            elif(max(self.p1.hp) < max(self.p2.hp)):
                #print("The second lowest wall P2 is stronger than P1. Player 2 win! [P1-P2: " + str(max(wall_p1)) + "-" + str(max(wall_p2)) + "]")
                return [1]
            else:
                # If is draw in same turn, player 2 who play behind win
                return [self.firstPlayer]
        elif(self.p1.hp.count(0) >= 2):
            #print("Player 1 Eliminated. Player 2 Win!")
            return [1]
        elif(self.p2.hp.count(0) >= 2):
            #print("Player 2 Eliminated. Player 1 Win!")
            return [0]
        ### if all card had been used and played
        elif(self.turnLimit > 0 and self.turn > self.turnLimit):
            ## compare by the lowest wall
            if(min(self.p1.hp) > min(self.p2.hp)):
                #print("The lowest wall P1 is stronger than P2. Player 1 win! [P1-P2: " + str(min(wall_p1)) + "-" + str(min(wall_p2)) + "]")
                return [0]
            elif(min(self.p1.hp) < min(self.p2.hp)):
                #print("The lowest wall P2 is stronger than P1. Player 2 win! [P1-P2: " + str(min(wall_p1)) + "-" + str(min(wall_p2)) + "]")
                return [1]
            else:
                ## compare by the median wall
                if(statistics.median(self.p1.hp) > statistics.median(self.p2.hp)):
                    #print("The second lowest wall P1 is stronger than P2. Player 1 win! [P1-P2: " + str(statistics.median(wall_p1)) + "-" + str(statistics.median(wall_p2)) + "]")
                    return [0]
                elif(statistics.median(self.p1.hp) < statistics.median(self.p2.hp)):
                    #print("The second lowest wall P2 is stronger than P1. Player 2 win! [P1-P2: " + str(statistics.median(wall_p1)) + "-" + str(statistics.median(wall_p2)) + "]")
                    return [1]
                else:
                    ## compare by the median wall
                    if(max(self.p1.hp) > max(self.p2.hp)):
                        #print("The second lowest wall P1 is stronger than P2. Player 1 win! [P1-P2: " + str(max(wall_p1)) + "-" + str(max(wall_p2)) + "]")
                        return [0]
                    elif(max(self.p1.hp) < max(self.p2.hp)):
                        #print("The second lowest wall P2 is stronger than P1. Player 2 win! [P1-P2: " + str(max(wall_p1)) + "-" + str(max(wall_p2)) + "]")
                        return [1]
                    else:
                        # If is draw in same turn, player 2 who play behind win
                        #print("Draw")
                        return [self.firstPlayer]
        else:   
            return False    


    ## Check if tile effect applied
    def isTileApplied(self, bot, effectCheck):
        for tile in self.tiles:
            if(bot["Location"] == tile["Location"]):
                if(effectCheck == tile["Name"] == "Fire" and bot["Player"] != tile["Player"]):
                    return tile["Param"]
                if(effectCheck == tile["Name"] == "Frozen" and bot["Player"] != tile["Player"]):
                    return tile["Param"]
                if(effectCheck == tile["Name"] == "Energy" and bot["Player"] == tile["Player"]):
                    return tile["Param"]
        return 0
            
    ## Check if location forward to enemy is occupied
    def getBot(self, playerName, location, spaceForward=0):
        for bot in self.spaces:
            if(playerName == self.p1.name and [location[0],location[1]+spaceForward] == bot["Location"]):
                return bot
            elif(playerName == self.p2.name and [location[0],location[1]-spaceForward] == bot["Location"]):
                return bot
            elif(spaceForward == 0 and location == bot["Location"]):
                return bot
        return False

    ## Check if location forward to enemy is occupied
    def getTile(self, location):
        for tile in self.tiles:
            if([location[0],location[1]] == tile["Location"]):
                return tile
        return False 
    
    ## get number of bot in each lane
    def getNumberOfBot(self, playerNames, lanes):
        number = 0
        for bot in self.spaces:
            if(bot["Player"] in playerNames and bot["Location"][0] in lanes):
                number = number + 1
        return number

    ## get order of bot in lane
    def getOrderOfBotInLane(self, playerName, location):
        list = []
        for bot in self.spaces:
            if(bot["Player"] == playerName and bot["Location"][0] == location[0]):
                list.append(bot["Location"][1])
    
        if(playerName == self.p1.name):
            list.sort()
        elif(playerName == self.p2.name):
            list.sort(reverse=True)
            
        return list.index(location[1])
    
    ## get the first bot in lane
    def getFrontestBotInLane(self, playerName, lane):
        list = []
        for bot in self.spaces:
            if(bot["Player"] == playerName and bot["Location"][0] == lane):
                list.append(bot["Location"][1])
    
        if(playerName == self.p1.name):
            list.sort()
        elif(playerName == self.p2.name):
            list.sort(reverse=True)

        if(len(list) > 0):
            return self.getBot(playerName, [lane,list[-1]], 0)
        else:
            return False

    ## check if it's in a turf
    def isInTurf(self, player, location):
        if(player.name == self.p1.name and location[1] < self.p1.ctrl[location[0]]):
            return True
        elif(player.name == self.p2.name and self.WIDTH - 1 - location[1] < self.p2.ctrl[location[0]]):
            return True
        else:
            return False

    def logState(self):
        img = self.getStateCNN()
        res = self.checkResult()

        if(self.turn == 1):
            self.gameTime = time.time();

        filename = str(self.gameTime) + "_" + str(self.turn);

        if(res == False):
            dir = "image/gameplay"
            img.save(dir +'/' + filename + '.bmp')
        elif(0 in res):
            dir = "image/gameplay"
            img.save(dir +'/' + filename + '.bmp')
            dir = "image/p1win"
            img.save(dir +'/' + filename + '.bmp')
        elif(1 in res):
            dir = "image/gameplay"
            img.save(dir +'/' + filename + '.bmp')
            dir = "image/p2win"
            img.save(dir +'/' + filename + '.bmp')

        if(res != False):
            filename = "Spellbot_Card.csv"
            # writing to csv file
            with open(filename, 'a') as csvfile:
                # creating a csv writer object
                csvwriter = csv.writer(csvfile)
                # writing the fields       
                if(os.stat(filename).st_size == 0):
                    csvwriter.writerow(self.getStateTitleSHAP())
                csvwriter.writerow(self.getStateSHAP())
        return

    ## write Stat
    def logStat(self):
        fileName = "SpellbotLog.txt"
        f = open(fileName, "a")
        # Header
        f.write("==Turn: " + str(self.turn) + "/" + str(self.turnLimit) + " Frame: "+ str(self.frame) +"\n")
        # Player 1
        f.write("-- "+ self.p1.name  + " --" + "\n") 
        f.write("HP: " + str(self.p1.hp[0]) + "," + str(self.p1.hp[1]) + "," + str(self.p1.hp[2]) + "\n")
        f.write("Mana: " + str(self.p1.mp) + "\n")
        f.write("Controls: " + str(self.p1.ctrl[0]) + "," + str(self.p1.ctrl[1]) + "," + str(self.p1.ctrl[2]) + "\n")
        f.write("Card in Deck: " + str(len(self.p1.deck)) + "\n")
        f.write("Card in Hand: \n")
        for cardIndex in self.p1.hand:
            obj = self.p1.getCard(cardIndex)
            f.write(str(obj.cardName) + " Mana: " + str(obj.mana) + " HP:" + str(obj.health) + " ATK:" + str(obj.attack) + " Ability:" + str(obj.desc) + "\n")
        # Player 2
        f.write("-- "+ self.p2.name  + " --" + "\n") 
        f.write("HP: " + str(self.p2.hp[0]) + "," + str(self.p2.hp[1]) + "," + str(self.p2.hp[2]) + "\n")
        f.write("Mana: " + str(self.p2.mp) + "\n")
        f.write("Controls: " + str(self.p2.ctrl[0]) + "," + str(self.p2.ctrl[1]) + "," + str(self.p2.ctrl[2]) + "\n")
        f.write("Card in Deck: " + str(len(self.p2.deck)) + "\n")
        f.write("Card in Hand: \n")
        for cardIndex in self.p2.hand:
            obj = self.p2.getCard(cardIndex)
            f.write(str(obj.cardName) + " Mana: " + str(obj.mana) + " HP:" + str(obj.health) + " ATK:" + str(obj.attack) + " Ability:" + str(obj.desc) + "\n")
        # Action
        f.write("-- ACTION --" + "\n") 
        for bot in self.moves:
            f.write(str(bot["Player"]) + " place " + str(bot["Name"]) + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + "\n")
        # Wipe moves history
        self.moves = []
        # Board
        f.write("-- BOARD (List View) --" + "\n")
        f.write("- Bot -" + "\n")
        for bot in self.spaces:
            f.write(str(bot["Player"]) + " place " + str(bot["Name"]) + " @ " + str(bot["Location"][0]) + "," + str(bot["Location"][1]) + "\n")
        f.write("- Tile -" + "\n")
        for tile in self.tiles:
            f.write(str(tile["Player"]) + " place " + str(tile["Name"]) + " @ " + str(tile["Location"][0]) + "," + str(tile["Location"][1]) + "\n")
        f.write("-- BOARD (Grid View) --" + "\n")
        for lane in range(3):
            # Player
            txt = str()
            for col in range(self.WIDTH):
                if(col == 0):
                    txt = txt + "".join("{:<13}".format(self.p1.name + "H:" + str(self.p1.hp[lane])))
                elif(col == self.WIDTH-1):
                    txt = txt + "".join("{:<13}".format(self.p2.name + "H:" + str(self.p2.hp[lane])))
                elif(self.getBot("def", [lane,col]) == False):
                    txt = txt + "".join("{:<13}".format("------------"))
                else:
                    txt = txt + "".join("{:<13}".format(self.getBot("def", [lane,col])["Player"]))
            f.write(txt)
            f.write("\n")
            # Player
            txt = str()
            for col in range(self.WIDTH):
                if(self.getBot("def", [lane,col]) == False):
                    txt = txt + "".join("{:<13}".format("------------"))
                else:
                    txt = txt + "".join("{:<13}".format(self.getBot("def", [lane,col])["Name"]))
            f.write(txt)
            f.write("\n")
            # Stats
            txt = str()
            for col in range(self.WIDTH):
                if(self.getBot("def", [lane,col]) == False):
                    txt = txt + "".join("{:<13}".format("------------"))
                else:
                    txt = txt + "".join("{:<13}".format("H:" + str(self.getBot("def", [lane,col])["Health"]) + "/A:" + str(self.getBot("def", [lane,col])["Attack"])))
            f.write(txt)
            f.write("\n")
            # Tile
            txt = str()
            for col in range(self.WIDTH):
                if(self.getTile([lane,col]) == False):
                    txt = txt + "".join("{:<13}".format("------------"))
                else:
                    txt = txt + "".join("{:<13}".format(self.getTile([lane,col])["Player"] + " " + self.getTile([lane,col])["Name"]))
            f.write(txt)
            f.write("\n")
            f.write("\n")
        f.close()
        return
             

## Section 2: Player Layer
This layer consists of the different player bots that compete against each other and generate the required game plays to train our model. This paper is using the Monte Cario Tree Search as an agent that explores future action spaces and game states, and then takes the best action.

This game class which is used as self-agent. You can define CSV file name that contains a set of card. These CSV file must in a same directory.

In [ ]:
from mcts_simple import *

class SpellBot(Game):
    WIDTH = 8
    TURN_LIMIT = 40
    
    def __init__(self):
        self.firstPlayer = random.randint(0, 1)
        self.orderOfPlayer = [self.firstPlayer,1-self.firstPlayer]
        self.currentOrder = 0
        self.players = [Player("P1","data_cardpack1.csv"),Player("P2","data_cardpack2.csv")] 
        self.board = Board(self.players[0], self.players[1])
        self.board.firstPlayer = self.firstPlayer;
        self.state = []
        self.rendered = False
        self.train = True

    def render(self):
        self.board.logState()
        return
        
    def get_state(self): 
        return self.board.getState()

    def number_of_players(self):
        return len(self.players)
    
    def current_player(self):
        return self.players[self.orderOfPlayer[self.currentOrder]]
    
    def possible_actions(self):
        return self.board.getAvaliableAction(self.players[self.orderOfPlayer[self.currentOrder]], self.board)
    
    def take_action(self, action):
        ## Extract
        cardIndex = action[0]
        location = (action[1],action[2])
        
        # Place Bot
        if(action[0] == action[1] == action[2] == -1):
            # If end turn, switch player and mark as acted
            self.players[self.orderOfPlayer[self.currentOrder]].acted = True
            self.currentOrder = int(1 - self.currentOrder)
        elif(cardIndex not in self.players[self.orderOfPlayer[self.currentOrder]].hand):
            ## Pass when no card exist(?)
            pass
        elif(action[1] == action[2] == -1):
            self.board.discardCard(self.players[self.orderOfPlayer[self.currentOrder]], cardIndex)
        else:
            self.board.placeBot(self.players[self.orderOfPlayer[self.currentOrder]], cardIndex, location)

        # If all players done, deal a damage
        if(self.players[0].acted == True and self.players[1].acted == True):
            self.board.proceed()
            self.rendered = True
            self.orderOfPlayer = [int(1 - x) for x in self.orderOfPlayer]
            self.board.firstPlayer = 1 - self.firstPlayer;
        return
    
    def has_outcome(self):
        return self.board.checkResult()

    def winner(self):
        # self.board.logStat()
        winners = self.board.checkResult()
        self.board.reset()
        return winners

## Section 3: Tournament Layer 
This section cover a iteration of tournament layer

1. Generate CSV file from parallel self-play

In [ ]:
game = SpellBot()
tree = OpenLoopUCT(game, training = True)
tree.self_play(iterations = 10000)

In [ ]:
tree.training = False
tree.self_play(iterations = 1)

## Section 4: Game Balance Layer 
Game balance layer

1. CNN for predict game. This part can demonstration win rate probability of a match using CNN and softmax function.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import os
from joblib import dump, load
from skimage.io import imread
from skimage.transform import resize

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

In [ ]:
data = dict()
data['label'] = []
data['data'] = []

for file in os.listdir('image/p1win'):
    if file[-3:] in {'bmp'}:
        im = imread(os.path.join('image/p1win', file))
        im = resize(im, (8, 27))
        data['label'].append('p1win')
        data['data'].append(im.flatten())

for file in os.listdir('image/p2win'):
    if file[-3:] in {'bmp'}:
        im = imread(os.path.join('image/p2win', file))
        im = resize(im, (8, 27))
        data['label'].append('p2win')
        data['data'].append(im.flatten())

In [ ]:
X = pd.DataFrame(data['data'])
y = pd.DataFrame(data['label'])

In [ ]:
data = dict()
data['filename'] = []
data['label'] = []
data['data'] = []

for file in os.listdir('image/gameplay'):
    if file[-3:] in {'bmp'}:
        im = imread(os.path.join('image/gameplay', file))
        im = resize(im, (8, 27))
        data['filename'].append(file)
        data['label'].append('p1win')
        data['data'].append(im.flatten())

In [ ]:
X = pd.DataFrame(data['data'])
y = pd.DataFrame(data['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layer_sizes':[8,8,8],
    'activation': ['relu'],
    'solver': ['adam'],
}

gridSearch = GridSearchCV(MLPClassifier(), param_grid, scoring='recall',verbose=2)
gridSearch.fit(X_train, y_train.replace({'p1win':0,'p2win':1}))

print('Score: ', gridSearch.best_score_)
print('Parameters: ', gridSearch.best_params_)

In [ ]:
clf = load('CNNwithSoftmax-model.joblib')

In [ ]:
predict_test = gridSearch.predict_proba(X_test)

In [ ]:
print(predict_test)

In [ ]:
predict_test = clf.predict_proba(X)

In [ ]:
import matplotlib.pyplot as plt

match_length = len(predict_test)
  
# create data
px = [int(i) for i in range(match_length)]
py1 = [predict_test[i][0] for i in range(match_length)]
py2 = [predict_test[i][1] for i in range(match_length)]
  
# plot lines
plt.plot(px, py1, label = "Player 1")
plt.plot(px, py2, label = "Player 2")
plt.legend()
plt.show()

2. SHAP for suggest game balance. This part can produce SHAP value of a set of games. 

In [ ]:
tree.training = False
tree.self_play(iterations = 1999)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

import shap

In [ ]:
df = pd.read_csv('Spellbot_Card.csv') 
print(df.shape)
header = list(df.columns.values)
df = df.drop('P1WIN', axis=1)
print(df.shape)
#print(header)
df.describe().transpose()

In [ ]:
target_column = ['P2WIN']
predictors = list(set(list(df.columns))-set(target_column))

In [ ]:
CX = df[predictors].values
Cy = df[target_column].values

CX_header = header.remove('P2WIN')

CX_train, CX_test, Cy_train, Cy_test = train_test_split(CX, Cy, test_size=0.30, random_state=40)
print(CX_train.shape); print(CX_test.shape)
print(Cy_train.shape); print(Cy_test.shape)

In [ ]:
Cmlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
Cmlp.fit(CX_train,Cy_train)

predict_train = Cmlp.predict(CX_train)
predict_test = Cmlp.predict(CX_test)

In [ ]:
dump(Cmlp, 'CNNwithSHAP-model.joblib') 

In [ ]:
header = list(df.columns.values)
header.remove('P2WIN')
explainer = shap.Explainer(Cmlp.predict, CX, feature_names = header)
shap_values = explainer(CX)

In [ ]:
shap.summary_plot(shap_values, color=shap_values, sort=False, plot_size = 0.25, max_display = 28)